## Day 24

In [281]:
from copy import deepcopy
from collections import defaultdict

def strenghtB(b,comp):
    s = 0
    for ic in b:
        s += sum(comp[abs(ic)])
    return s

def buildBridges(comp_,npurge=10000000):

    comp = deepcopy(comp_)
    
    # find [0/X] components 
    starts = []
    for c in comp:
        if c[0] == 0:
            starts.append(c)

    # place them at begining of comp array to avoid components with index 0
    for s in starts:
        comp.pop(comp.index(s))
        comp = [s]+comp
        if c[0] == 0:
            starts.append(c)

    # initialize bridges
    bridges = [ [ comp.index(s)] for s in starts ]

    # build bridges as sequence of component indexes. Negative indexes indicates reversed direction
    while True:
        #print(bridges)
        bridges_new = []
        for b in bridges:
            for ic in range(len(comp)):
                bnew = list(b)
                if ic not in b and -ic not in b:
                    if bnew[-1]>=0:
                        if comp[bnew[-1]][1] == comp[ic][0]:
                            bnew.append(ic)
                        elif comp[bnew[-1]][1] == comp[ic][1]:
                            bnew.append(-ic)
                    elif bnew[-1]<0:
                        if comp[-bnew[-1]][0] == comp[ic][1]:
                            bnew.append(-ic)
                        elif comp[-bnew[-1]][0] == comp[ic][0]:
                            bnew.append(ic)
                if bnew not in bridges_new: 
                    bridges_new.append(bnew)
                    
        # I should find an heuristic to prune the bridges to avoid explosing the queue!
        #print("N_bridges =",len(bridges_new))
        #if len(bridges) == len(bridges_new):
        #    return bridges_new, comp # returns also the rearranged component list
        #bridges = deepcopy(bridges_new)
        
        # Idea: computing the strenghs for the current bridge selection, and only keep the npurge bridges 
        # with highest streght. This might work if npurge is large enough, e.g. keeping enough low-strenght 
        # segments that could stil evolve toward stronger ones. I should also test how the result depends 
        # on npurge
        strenghts = [strenghtB(b,comp) for b in bridges_new]
        bridges_sel = []
        if len(strenghts)>=npurge:
            select = sorted(strenghts)[-npurge:]
            for s in select:
                bridges_sel.append(bridges_new[strenghts.index(s)])
        else:
            bridges_sel = deepcopy(bridges_new)
        if len(bridges) == len(bridges_sel):
            return bridges_sel, comp 
        bridges = deepcopy(bridges_sel)

def maxStrenght(comp,npurge=1_000_000):
    bridges,comp_r = buildBridges(comp,npurge) 
    return max([strenghtB(b,comp_r) for b in bridges])

In [301]:
lines = [
    '0/2',
    '2/2',
    '2/3',
    '3/4',
    '3/5',
    '0/1',
    '10/1',
    '9/10'  
]

comp_test = [ [ int(q) for q in l.split("/") ] for l in lines ] 
maxstr_test = maxStrenght(comp_test)
print("Test 1. Max strenght =",maxstr_test)

Test 1. Max strenght = 31


In [308]:
with open('data/input24.txt') as f:
    comp = [ [ int(q) for q in l.split("/") ] for l in f.readlines() ]

maxstr = maxStrenght(comp,npurge=3000) # some stydy of dependence of result on npurge was necessary :-)
print("Part 1. Max strenght =",maxstr)

Part 1. Max strenght = 1859


In [304]:
def longestBridges(comp_,nprune=1_000_000):

    comp = deepcopy(comp_)
    
    # find [0/X] components 
    starts = []
    for c in comp:
        if c[0] == 0:
            starts.append(c)

    # place them at begining of comp array to avoid components with index 0
    for s in starts:
        comp.pop(comp.index(s))
        comp = [s]+comp
        if c[0] == 0:
            starts.append(c)

    # initialize bridges
    bridges = [ [ comp.index(s)] for s in starts ]

    # build bridges as sequence of component indexes. Negative indexes indicates reversed direction
    while True:
        #print(bridges)
        bridges_new = []
        for b in bridges:
            for ic in range(len(comp)):
                bnew = list(b)
                if ic not in b and -ic not in b:
                    if bnew[-1]>=0:
                        if comp[bnew[-1]][1] == comp[ic][0]:
                            bnew.append(ic)
                        elif comp[bnew[-1]][1] == comp[ic][1]:
                            bnew.append(-ic)
                    elif bnew[-1]<0:
                        if comp[-bnew[-1]][0] == comp[ic][1]:
                            bnew.append(-ic)
                        elif comp[-bnew[-1]][0] == comp[ic][0]:
                            bnew.append(ic)
                if bnew not in bridges_new: 
                    bridges_new.append(bnew)

        # pruning shortest bridges
        lenghts = [ len(b) for b in bridges_new ]
        bridges_long = []
        for il in range(len(lenghts)):
            if lenghts[il]==max(lenghts):
                bridges_long.append(bridges_new[il])
        
        # apply same pruning based on strenght as in Part 1 to speed up convergence
        strenghts = [strenghtB(b,comp) for b in bridges_long]
        bridges_sel = []
        if len(strenghts)>=npurge:
            select = sorted(strenghts)[-npurge:]
            for s in select:
                bridges_sel.append(bridges_long[strenghts.index(s)])
        else:
            bridges_sel = deepcopy(bridges_long)
        
        if len(bridges) == len(bridges_sel):
            return bridges_sel, comp
        bridges = deepcopy(bridges_sel)

In [305]:
longest_test,comp_r_test = longestBridges(comp_test)
print("Test 2. Strenght longest =",max([ strenghtB(b,comp_r_test) for b in longest_test ]))

Test 2. Strenght longest = 19


In [306]:
longest,comp_r = longestBridges(comp,3000)
print("Part 2. Strenght longest =",max([ strenghtB(b,comp_r) for b in longest ]))

Part 2. Strenght longest = 1799
